Hidden markov model - forward backward approach for text prediction

Text - corpus on which text prediction done is playerlines for different plays.
The assumption made here is let's say we want to write a play and predict the best possible playerlines from the existing playerlines of each play.

Input dataset : https://www.kaggle.com/kingburrito666/shakespeare-plays
        
References : https://en.wikipedia.org/wiki/Forward%E2%80%93backward_algorithm

In [1]:
#importing necessary libraries

import pandas as pd
import numpy as np

In [2]:
#Reading file Shakespeare_data

Path = r'C:\Users\Dhwani\Desktop\ML\HW2\Shakespeare_data\Shakespeare_data.csv'
ShakespeareDf = pd.read_csv(Path)

In [3]:
ShakespeareDf[['Play','PlayerLine']].head()

,Play,PlayerLine
0,Henry IV,ACT I
1,Henry IV,SCENE I. London. The palace.
2,Henry IV,"Enter KING HENRY, LORD JOHN OF LANCASTER, the ..."
3,Henry IV,"So shaken as we are, so wan with care,"
4,Henry IV,"Find we a time for frighted peace to pant,"


In [4]:
# Taking sample of 2 playerlines from each of the 10 plays

playerLines = [] 
playL ={}
UniquePlay = ShakespeareDf.Play.unique()[0:10]
for play in UniquePlay :
    playerLines.append(ShakespeareDf[['Play','PlayerLine']][ShakespeareDf['Play'] == play][10:12])

In [5]:
#importing natural language tool kit for natural language processing
import nltk
import string

tokens={}
#this function tokenizes a sentence into number of words.So tokenizing each word of playerline
def tokenize(text):
    sentence = text.rstrip().lower()
    return sentence.translate(str.maketrans('','', string.punctuation)).split()

In [6]:
for lines in playerLines:
      for sentence in lines.PlayerLine:
            print(sentence.translate(str.maketrans('','', string.punctuation)))

Nor bruise her flowerets with the armed hoofs
Of hostile paces those opposed eyes
England neer had a king until his time
Virtue he had deserving to command
Seven earls twelve barons and twenty reverend bishops
I have performd my task and was espoused
Charged our main battles front and breaking in
Were by the swords of common soldiers slain
worthiness would stir it up where it wanted rather
than lack it where there is such abundance
properly stays me here at home unkept for call you
that keeping for a gentleman of my birth that
The buckles on his breast reneges all temper
And is become the bellows and the fan
Who wanting guilders to redeem their lives
Have seald his rigorous statutes with their bloods
Ist a verdict
No more talking ont let it be done away away
Unto a poor but worthy gentleman shes wedded
Her husband banishd she imprisond all


In [7]:
#Calling tokenize function on playerlines

tokens=[]
for lines in playerLines:
      for sentence in lines.PlayerLine:
        tempToken = tokenize(sentence)
        tokens.append(tempToken)

In [8]:
# result of tokenization
tokens[5]

['i', 'have', 'performd', 'my', 'task', 'and', 'was', 'espoused']

In [9]:
# saving words in prev_word - word pair
def addingTextInDictionary(dictionary, key, value):
    if key not in dictionary:
        dictionary[key] = []
    dictionary[key].append(value)

In [10]:
#Adding words into dictionary and counting occurences of next word count
transition_probability = {}
start_word_probability = {}
for tokensPerSentence in tokens:
 
 tokens_length = len(tokensPerSentence)
 for i in range(tokens_length):
    token = tokensPerSentence[i]
    
    prev_token = tokensPerSentence[i - 1]
    if i == 0:
        start_word_probability[token] = start_word_probability.get(token, 0) + 1
        continue;
    else:
          addingTextInDictionary(transition_probability, prev_token, token)
          if i == tokens_length  - 1:
           addingTextInDictionary(transition_probability, token, 'END')

In [11]:
# Computing transition and initial probability based on counts. In short putting weights for the word pair
dictWithProbability = {}
count_start_word = sum(start_word_probability.values())
for key,prob in start_word_probability.items():
    start_word_probability[key] = prob / count_start_word
    
for prev_word,word in transition_probability.items():
  dictWithProbability = {}
  total = len(word) #total next words for a token
  for item in word:
   dictWithProbability[item] = dictWithProbability.get(item,0) + 1
  for key,value in dictWithProbability.items():
   dictWithProbability[key] = value / total
  transition_probability[prev_word] = dictWithProbability


In [12]:
transition_probability

{'nor': {'bruise': 1.0},
 'bruise': {'her': 1.0},
 'her': {'flowerets': 0.5, 'husband': 0.5},
 'flowerets': {'with': 1.0},
 'with': {'the': 0.5, 'their': 0.5},
 'the': {'armed': 0.2,
  'swords': 0.2,
  'buckles': 0.2,
  'bellows': 0.2,
  'fan': 0.2},
 'armed': {'hoofs': 1.0},
 'hoofs': {'END': 1.0},
 'of': {'hostile': 0.3333333333333333,
  'common': 0.3333333333333333,
  'my': 0.3333333333333333},
 'hostile': {'paces': 1.0},
 'paces': {'those': 1.0},
 'those': {'opposed': 1.0},
 'opposed': {'eyes': 1.0},
 'eyes': {'END': 1.0},
 'england': {'neer': 1.0},
 'neer': {'had': 1.0},
 'had': {'a': 0.5, 'deserving': 0.5},
 'a': {'king': 0.25, 'gentleman': 0.25, 'verdict': 0.25, 'poor': 0.25},
 'king': {'until': 1.0},
 'until': {'his': 1.0},
 'his': {'time': 0.3333333333333333,
  'breast': 0.3333333333333333,
  'rigorous': 0.3333333333333333},
 'time': {'END': 1.0},
 'virtue': {'he': 1.0},
 'he': {'had': 1.0},
 'deserving': {'to': 1.0},
 'to': {'command': 0.5, 'redeem': 0.5},
 'command': {'END':

In [13]:
# probability of initial words 

start_word_probability['nor']

0.05

In [14]:
tlength = 0
for tokensPerSentence in tokens:
 #print(tokensPerSentence)
 tlength = tlength + len(tokensPerSentence)

In [15]:
# total number of tokens generated hence the hidden states
tlength

154

Emission probability

For emission probability, we have considered plays for each playerline. Hidden states are the tokens generated from text-corpus

In [16]:
import re

def findWholeWord(w):
    return re.compile(r'\b({0})\b'.format(w), flags=re.IGNORECASE).search


In [17]:
states=[]
for i in tokens:
    for j in i:
        states.append(j)
states = set(states)
#states

In [18]:
observations = []#ALl plays
emission_probability = {}
for i in states:
   for lines in playerLines:
     for sentence in lines.PlayerLine:
       if findWholeWord(i)(sentence.translate(str.maketrans('','', string.punctuation))) != None :
         playl = lines.Play.unique()
         observations.append(playl[0])
         addingTextInDictionary(emission_probability, i, playl[0])
observations = set(observations)
observations = tuple(observations)

In [19]:
dictWithProbability = {}
for prev_word,word in emission_probability.items():
  dictWithProbability = {}
  total = len(word) #total next words for a token
  for item in word:
   dictWithProbability[item] = dictWithProbability.get(item,0) + 1
  for key,value in dictWithProbability.items():
   dictWithProbability[key] = value / total
  emission_probability[prev_word] = dictWithProbability


In [20]:
# Computing weights of words for the playerlines per play

prev_prob={}
emi_zero={}
for key,value in emission_probability.items():
    for obs in observations:
     prev_prob[obs] = 0.0
     emi_zero[obs] = 0.0
    for k,v in value.items() :
     for obs in observations:
       if k == obs:
        emi_zero[obs] = v
       else:
        if prev_prob[obs] == 0.0:
          emi_zero[obs] = 0.0
        else :
          emi_zero[obs] = prev_prob[obs]
        prev_prob = emi_zero
    emission_probability[key].update(emi_zero)
print(emission_probability)

{'away': {'Coriolanus': 1.0, 'A Comedy of Errors': 0.0, 'Henry IV': 0.0, 'Alls well that ends well': 0.0, 'Henry VI Part 1': 0.0, 'Cymbeline': 0.0, 'Henry VI Part 2': 0.0, 'As you like it': 0.0, 'Antony and Cleopatra': 0.0, 'Henry VI Part 3': 0.0}, 'earls': {'Henry VI Part 2': 1.0, 'A Comedy of Errors': 0.0, 'Henry IV': 0.0, 'Alls well that ends well': 0.0, 'Henry VI Part 1': 0.0, 'Cymbeline': 0.0, 'As you like it': 0.0, 'Antony and Cleopatra': 0.0, 'Henry VI Part 3': 0.0, 'Coriolanus': 0.0}, 'become': {'Antony and Cleopatra': 1.0, 'A Comedy of Errors': 0.0, 'Henry IV': 0.0, 'Alls well that ends well': 0.0, 'Henry VI Part 1': 0.0, 'Cymbeline': 0.0, 'Henry VI Part 2': 0.0, 'As you like it': 0.0, 'Henry VI Part 3': 0.0, 'Coriolanus': 0.0}, 'all': {'Antony and Cleopatra': 0.5, 'Cymbeline': 0.5, 'A Comedy of Errors': 0.0, 'Henry IV': 0.0, 'Alls well that ends well': 0.0, 'Henry VI Part 1': 0.0, 'Henry VI Part 2': 0.0, 'As you like it': 0.0, 'Henry VI Part 3': 0.0, 'Coriolanus': 0.0}, 'you'

Algorithm for forward - backward apporach works as :
1. It computes probability of each state based on the emission probabilty and transition probability in forward direction(0 to t-1 for t)
2. Does the same in backward direction from t+1.
3. Smoothening and filtering out errors

In [42]:
# Forward - backward algorithm HMM

def fwd_bkw(observations, states, start_prob, trans_prob, emm_prob, end_st):
    # forward part of the algorithm
    fwd = []
    f_prev = {}
    for kv in start_word_probability.items():
      default_start = kv[1]
      break
    for i, observation_i in enumerate(observations):
        f_curr = {}
        for st in states:
            if i == 0:
                # base case for the forward part
              if st in start_word_probability:
                  prev_f_sum = start_word_probability[st]
              else:
                  prev_f_sum = default_start
            else:
                for k in states:
                    for key,value in trans_prob[k].items():
                      prev_f_sum = prev_f_sum + (f_prev[k]*value)
            f_curr[st] = emm_prob[st][observation_i] * prev_f_sum

        fwd.append(f_curr)
        f_prev = f_curr
    fwd_total = 0
    for k in states:
            for key,value in trans_prob[k].items():
              if end_state in trans_prob[k]:
               fwd_total = fwd_total + (f_curr[k] * trans_prob[k][end_st])
    p_fwd = fwd_total

    # backward part of the algorithm
    bkw = []
    b_prev = {}
    for i, observation_i_plus in enumerate(reversed(observations[1:]+(None,))):
        b_curr = {}
        for st in states:
            if i == 0:
                # base case for backward part
                trans = transition_probability[st]
                if end_state in trans:
                 b_curr[st] = transition_probability[st][end_state]
                else:
                 b_curr[st] = 1.0
            else:
                for l in states:
                    total = 0
                    for key,value in trans_prob[l].items():
                      total = total + value * emm_prob[l][observation_i_plus] * b_prev[l]
                      b_curr[st] = total

        bkw.insert(0,b_curr)
        b_prev = b_curr

    for l in states:
        total_bkw = 0
        for key,value in trans_prob[l].items():
          total_bkw = total_bkw + (b_curr[st] * emm_prob[l][observations[0]] * b_curr[l])
    p_bkw = total_bkw

    # merging the two parts
    posterior = []
    for i in range(len(observations)):
        posterior.append({st: fwd[i][st] * bkw[i][st] / p_fwd for st in states})

    
    return fwd, bkw, posterior

In [43]:
fwd = []
bkw = []
posterior = []
end_state = 'END'
fwd,bkw,posterior = fwd_bkw(observations, states, start_word_probability, transition_probability, emission_probability, end_state)

In [50]:
# Generating text based on the weights in posterior for each word. The observation sequence is like 
#plays starting from 1 to n. So predicting text based on that.


text_series = []
for line in fwd :
    for key,value in line.items():
      if value > 0 :
        text_series.append(key)
print('Text prediction and playerline for our new play in forward path is :')
print(' '.join(text_series))

Text prediction and playerline for our new play in forward path is :
bloods seald redeem his lives wanting who statutes have to with rigorous guilders their opposed those armed hostile hoofs her eyes the paces with bruise flowerets nor of abundance is such rather worthiness would there it stir wanted lack than where up his command england deserving until had to virtue time he king neer a all wedded gentleman but banishd shes her husband imprisond poor worthy unto she a earls i barons twenty was reverend and my have espoused performd twelve bishops task seven you for properly gentleman me stays my here home that unkept keeping birth call of at a become all is reneges on his bellows temper fan and buckles breast the main common and battles our charged in the slain were by soldiers breaking swords of front away done ist more verdict no let it talking ont be a


In [49]:
# Generating text based on the weights in posterior for each word. The observation sequence is like 
#plays starting from 1 to n. So predicting text based on that.


text_series = []
for line in bkw :
    for key,value in line.items():
      if value > 0 :
        text_series.append(key)
print('Text prediction and playerline for our new play in backward path is :')
print(' '.join(text_series))

Text prediction and playerline for our new play in backward path is :
away earls become all you i bloods abundance is for wedded reneges opposed seald main redeem properly those common on done his barons twenty was reverend bellows such temper fan ist gentleman armed but rather more hostile banishd and buckles lives verdict battles me command wanting england stays my worthiness shes hoofs no our here who home would deserving charged that her statutes let unkept until in there it stir had have to wanted eyes breast the paces espoused talking with virtue time lack slain husband than ont imprisond he rigorous were king performd twelve poor worthy by bishops task seven soldiers bruise where breaking flowerets up keeping birth swords unto she neer call nor of at guilders front be their a away earls become all you i bloods abundance is for wedded reneges opposed seald main redeem properly those common on done his barons twenty was reverend bellows such temper fan ist gentleman armed but rath

In [48]:
# Generating text based on the weights in posterior for each word. The observation sequence is like 
#plays starting from 1 to n. So predicting text based on that.


text_series = []
for line in posterior :
    for key,value in line.items():
      if value > 0 :
        text_series.append(key)
print('Text prediction and playerline for our new play is :')
print(' '.join(text_series))

Text prediction and playerline for our new play is :
main common and battles our charged in the slain were by soldiers breaking swords of front away done ist more verdict no let it talking ont be a


Conclusion : Text prediction is based on the sum of weights of text from previous markov chain. The error rates are reduced in case of smoothening case which gives better text prediction. From our result, we can see that prediction for the posterior weights is better.